<a href="https://colab.research.google.com/github/makhalid1999/contrastive-unpaired-translation/blob/master/achieving_depth_realism.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Achieving Depth Realism**

Installing the required packages

In [ ]:
!pip install visdom

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for visdom: filename=visdom-0.2.4-py3-none-any.whl size=1408196 sha256=0a46e102843ace49cb83741890f46c73ef0c00e7f39f782daba243e441a5e1c5
  Stored in directory: /root/.cache/pip/wheels/42/29/49/5bed207bac4578e4d2c0c5fc0226bfd33a7e2953ea56356855
Successfully built visdom


In [ ]:
!pip install dominate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install torch-fidelity
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 14.2 MB/s eta 0:00:00


Cloning the github repository for Contrastive Unpaired Translation

In [ ]:
!git clone https://github.com/makhalid1999/contrastive-unpaired-translation.git

Cloning into 'contrastive-unpaired-translation'...
remote: Enumerating objects: 298, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 298 (delta 10), reused 1 (delta 0), pack-reused 278
Receiving objects: 100% (298/298), 17.92 MiB | 14.95 MiB/s, done.
Resolving deltas: 100% (152/152), done.


Running the code with our data and parameters

In [ ]:
!python /content/contrastive-unpaired-translation/train.py --name real2fake_l --dataroot '/content/drive/MyDrive/ML_DATA/Project data' --input_nc 1 --output_nc 1 --CUT_mode FastCUT  --lambda_GAN 3 #--ngf 192 #--ndf 128 --netD 'pixel'

----------------- Options ---------------
                 CUT_mode: FastCUT                       	[default: CUT]
               batch_size: 1                             
                    beta1: 0.5                           
                    beta2: 0.999                         
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                crop_size: 256                           
                 dataroot: /content/drive/MyDrive/ML_DATA/Project data	[default: placeholder]
             dataset_mode: unaligned                     
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: None                          
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://loc

Getting the results

In [ ]:
!python '/content/contrastive-unpaired-translation/test.py' --name real2fake_l --dataroot '/content/drive/MyDrive/ML_DATA/Project data' --input_nc 1 --output_nc 1 #--ngf 192 #--ndf 128 --netD 'pixel'

----------------- Options ---------------
                 CUT_mode: CUT                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: /content/drive/MyDrive/ML_DATA/Project data	[default: placeholder]
             dataset_mode: unaligned                     
                direction: AtoB                          
          display_winsize: 256                           
               easy_label: experiment_name               
                    epoch: latest                        
                     eval: False                         
        flip_equivariance: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: xavier                        
                 input_nc: 1                        

In [ ]:
import os
files = os.listdir('/content/results/real2fake_l/test_latest/images/fake_B')
import torch
from PIL import Image
import numpy as np
images = []
for file in files:
  image = Image.open('/content/results/real2fake_l/test_latest/images/fake_B/' + file).convert("F")
  image = np.asarray(image)
  image = torch.from_numpy(image).unsqueeze(0).float()
  images.append(image)
images = torch.stack(images)
print(images.shape)

torch.Size([47, 1, 256, 256])


Importing the images in a vector format for metric calculations

In [ ]:
import os
files = os.listdir('/content/drive/MyDrive/ML_DATA/Project data/nyu/office_real/nyu_office_1_out')
images_r = []
for file in files[0:47]:
  image_r = Image.open('/content/drive/MyDrive/ML_DATA/Project data/nyu/office_real/nyu_office_1_out/' + file).convert("F")
  image_r = np.asarray(image_r)
  image_r = torch.from_numpy(image_r).unsqueeze(0).float()
  images_r.append(image_r)
images_r = torch.stack(images_r)[0:50]

In [ ]:
import os
files = os.listdir('/content/results/real2fake_l/test_latest/images/real_B/')
images_f = []
for file in files[0:47]:
  image_f = Image.open('/content/results/real2fake_l/test_latest/images/real_B/' + file).convert("F")
  image_f = np.asarray(image_f)
  image_f = torch.from_numpy(image_f).unsqueeze(0).float()
  images_f.append(image_f)
images_f = torch.stack(images_f)[0:50]

Finding the Frechet Inception Distance

In [ ]:
from torchmetrics.image.fid import FrechetInceptionDistance
fid = FrechetInceptionDistance(feature=64)
fid.update(torch.cat((images_f, images_f, images_f), 1)[:, :, :256, :256].type(torch.uint8), real = True)
fid.update(torch.cat((images_r, images_r, images_r), 1)[:, :, :256, :256].type(torch.uint8), real = False)
fid.compute()

tensor(6.8698)